# Measurement quality from affordable GNSS receivers

In this section we will assess some basic differences in terms of GNSS observables
between a geodetic-grade and an affordable receiver.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from roktools import rinex
from roktools.gnss.types import ConstellationId

# Instructions to import code within the custom source folder
import sys
sys.path.append('../source/')

%matplotlib widget

Using the [`roktools` library](https://pypi.org/project/roktools/), the RINEX data from the receivers will be stored
into a `pandas` `DataFrame` (`df` for short) for manipulation.

In [ ]:
# DataFrame with data from a geodetic receiver
df_geodetic = rinex.to_dataframe('../assets/SEYG00SYC_R_20140581500_05H_01S_MO.rnx')

In [ ]:
# DataFrame with an affordable receiver
df_afford = rinex.to_dataframe('../assets/MTIC00ESP_R_20191221131_05H_01S_MO.rnx')

The `DataFrame` can be consider a *CSV* file of sorts, where each row has a time tag, satellite, observables and other fields that will be explained and used later in the notebook. A preview of the contents can be obtained with the `head` method

In [ ]:
df_geodetic.head()

## Observable types

Once we have loaded the RINEX files into DataFrames, we can perform some
basic checks on the differences between geodetic and affordable GNSS data.

The following example gives you *the channels tracked by the receiver for each GNSS constellation*.

The channel corresponds to the last two characters (i.e. band and attribute) of 
the [RINEX observation code](https://files.igs.org/pub/data/format/rinex_4.00.pdf) (see Section 5.2.17). 
For instance `1C` for GPS means the observables obtained with the C/A tracking at the L1 frequency.

To know the observables for constellation, we will use the `groupby` method of `pandas`.

In [ ]:
# We will group the data using various criteria
columns = ['constellation', 'channel']

# Use groupby() to group by the two columns and apply unique()
unique_combinations = df_geodetic.groupby(columns).size()

# Print the unique combinations (along with the number of samples for each
# tracking channel)
print(unique_combinations)

We can now perform the same for the affordable receivers

In [ ]:
unique_combinations = df_afford.groupby(columns).size()

print(unique_combinations)

In these examples, the following basic differences are observed:

- the geodetic receiver tracks various frequencies (GPS L1/L2/L5, Galileo E1/E5a/E5b/E5, ...) whereas the affordable receiver tracks typically **two frequencies** (GPS L1/L2, Galileo E1/E5b, ...)
- the affordable receiver does not attempt to track encrypted codes (i.e. GPS `P` code) by means of e.g. z-tracking loops. **Only civilian codes** (e.g. GPS L2C) are used.

Some other strenghts of affordable receivers:

- Availability of SNR and Doppler measurements (not always available in 30s or high rate CORS data)
- High rate up to 0.1s (or even higher) available for affordable measurements



## Code noise: Detrended code-minus-carrier

The observable code noise of a GNSS receiver can be estimated using the 
[code-minus-carrier combination](cmc_combination). This section illustrates
how to estimate to check some basic differences between receiver types

First we will import several methods that will be required to process data
(will be illustrated later)

In [ ]:
# Import the modules from the custom 'source' package of this book
from gnss.observables import compute_code_minus_carrier
from gnss.edit import mark_time_gap, detrend, remove_mean

The steps to be followed to compute the unbiased detrended Code minus carrier
are detailed in the following steps (taking as example the Geodetic receiver
and then applying it to the Affordable receiver)

1. *Edit* the data and find phase breaks such as cycle
slips. In this example, since receivers already provide with Loss-of-Lock Indicator
(LLI), we will only mark phase breaks due to data **time gaps**

In [ ]:
df_geodetic_cmc = mark_time_gap(df_geodetic)

2. Proceed to **compute the code minus carrier (CMC)** observable. Each row of the `DataFrame`
contains the range and phase, so there will be as many CMC observables as rows.
A note of caution: the phase is usually expressed in cycles, therefore we will
need to get the wavelength from the tracking channel in order to be consistent
with the units

In [ ]:
df_geodetic_cmc = compute_code_minus_carrier(df_geodetic_cmc)

In [ ]:
from helpers import compute_elapsed_seconds

# Plot a CMC for a channel and satellite
signal = 'E121X'
df_sample = df_geodetic_cmc[df_geodetic_cmc['signal'] == signal]
plt.close()
plt.plot(compute_elapsed_seconds(df_sample['epoch'])/3600, df_sample['cmc'], ',k')
plt.xlabel('Elapsed hours')
plt.ylabel('CMC [m]')
_ = plt.title(f'Code-minus-carrier for {signal}')


3. Because the CMC contains twice the ionosphere, which is a nuisance parameter
for us (because we are interested in the noise). We will proceed to remove its
contribution with a simple detrending, based on a rolling window

In [ ]:
n_samples = 5
df_geodetic_cmc = detrend(df_geodetic_cmc, 'cmc', n_samples)

In [ ]:
# Plot the detrened CMC for a channel and satellite
signal = 'E121X'
df_sample = df_geodetic_cmc[df_geodetic_cmc['signal'] == signal]
plt.close()
plt.plot(compute_elapsed_seconds(df_sample['epoch'])/3600, df_sample['cmc_detrended'], ',k')
plt.xlabel('Elapsed hours')
plt.ylabel('Detrended CMC [m]')
plt.ylim(-1, 1)
_ = plt.title(f'Detrended Code-minus-carrier for {signal}')


In [ ]:

# Analysis for Ublox receiver
df_afford_cmc = mark_time_gap(df_afford)
df_afford_cmc = compute_code_minus_carrier(df_afford_cmc)
df_afford_cmc = detrend(df_afford_cmc, 'cmc', n_samples)


Let's compute now an estimate of the code noise for the geodetic grade and
the affordable receiver, for a specific band and constellation.

In [ ]:
# GLONASS is excluded as the slot number to know the frequency (and hence the wavelength)
# is missing
condition1 = df_geodetic_cmc['constellation'] == ConstellationId.GPS
condition2 = df_geodetic_cmc['channel'] == '2W'
#condition2 = df_geodetic_cmc['channel'] == '1C'
df_tmp_g = df_geodetic_cmc[condition1 & condition2]

condition1 = df_afford_cmc['constellation'] == ConstellationId.GPS
condition2 = df_afford_cmc['channel'] == '2L'
#condition2 = df_afford_cmc['channel'] == '1C'
df_tmp_a = df_afford_cmc[condition1 & condition2]

noise_samples_geodetic = df_tmp_g['cmc_detrended']
noise_samples_afford = df_tmp_a['cmc_detrended']

rms_geodetic = np.sqrt(np.mean(np.square(noise_samples_geodetic)))
rms_afford = np.sqrt(np.mean(np.square(noise_samples_afford)))

print(f'Geodetic   receiver code noise: {rms_geodetic:.2} m')
print(f'Affordable receiver code noise: {rms_afford:.2} m')

## Observable noise: Detrended LI

A basic quality parameter we can check is the computation of the detrended ionospheric observable
observation. The ionospheric combination removes the geometry, while the detrending process 
removes the ionosphere as well as the biases (ambiguities and hardware biases).
The remaining terms is an approximation of the phase noise (if LI is used) 
or the code noise (if PI is used instead)

In [ ]:
import pandas as pd
from roktools.gnss.types import ConstellationId, TrackingChannel

def compute_cmc(df: pd.DataFrame, constellation: ConstellationId, channel_a: TrackingChannel, channel_b: TrackingChannel) -> pd.DataFrame:
    """
    Compute the Code minus Carrier
    """

    # Create subsets of the DataFrame corresponding to the constellation and each of
    # the channels selected to build the ionospheric combination
    df_a = df[(df['constellation'] == constellation) & (df['channel'] == str(channel_a))]
    df_b = df[(df['constellation'] == constellation) & (df['channel'] == str(channel_b))]

    # Compute the wavelength of the two tracking channels
    wl_a = channel_a.get_wavelength(constellation)
    wl_b = channel_b.get_wavelength(constellation)
    
    # Use merge to join the two tables
    df_out = pd.merge(df_a, df_b, on=['epoch', 'sat'], how='inner', suffixes=('_a', '_b'))
    df_out['li_m'] = df_out['phase_a'] * wl_a - df_out['phase_b'] * wl_b
    df_out['pi_m'] = df_out['range_b'] - df_out['range_a']

    # Define a custom rolling function to calculate the trend
    def do_rolling_window(group, column, new_column):
        group[new_column] = group[column].rolling(window=30).mean()
        return group

    # Apply the custom rolling function within each group
    df_tmp = df_out.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'li_m', 'li_trend_m'))
    b = df_tmp.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'pi_m', 'pi_trend_m'))

    return b

def get_seconds_since_start(epochs:pd.Series) -> list:
    """
    Converts a pandas series of epochs into a list of secodf.head()nds relative to the first epoch
    """
    return (epochs - epochs[0]).dt.total_seconds().tolist()
